In [1]:
# %%
import petsc4py
from petsc4py import PETSc
import os
os.environ["UW_TIMING_ENABLE"] = "1"

## --- Underworld

import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function
from underworld3 import timing

import numpy as np
import sympy

from underworld3.coordinates import CoordinateSystem, CoordinateSystemType
from underworld3.cython import petsc_discretisation




In [2]:
n_els = 4
mesh = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0), maxCoords=(1.0, 1.0), cellSize=1 / n_els, 
    qdegree=3, refinement=3
)

mesh.dmVars = mesh.dm.clone()

In [3]:
class meshVarProto(uw._api_tools.uw_object):
    """
    Mesh Variable Rewrite Test Classs
    """

    def __init__(self, mesh, name):

        import re

        # Notes:
        #    holding a local dm for the variable


        self.mesh = mesh
        self.name = name
        self.clean_name = re.sub(r"[^a-zA-Z0-9_]", "", name)

        self.dm = self.mesh.dm.clone()
        self.num_components = 2

        dim = self.mesh.dim

        
        petsc_fe = PETSc.FE().createDefault(
            dim,
            self.num_components,
            self.mesh.isSimplex,
            self.mesh.qdegree,
            name + "_",
            PETSc.COMM_WORLD,
        )

        # global
        # self.field_id = self.mesh.dm.getNumFields()
        # self.mesh.dm.setField(self.field_id, petsc_fe)
        # field, _ = self.mesh.dm.getField(self.field_id)
        # field.setName(self.clean_name)

        # local
        self.field_id = self.dm.getNumFields()
        self.dm.setField(self.field_id, petsc_fe)
        field, _ = self.dm.getField(self.field_id)
        field.setName(self.clean_name)

        self._lvec = self.dm.createLocalVec()
        
        

        return

In [4]:
meshVar1 = meshVarProto(mesh, "testVar1")
meshVar2 = meshVarProto(mesh, "testVar2")
meshVar3 = meshVarProto(mesh, "testVar3")

In [5]:
mesh.dmVars.addField(meshVar1.dm.getField(meshVar1.field_id)[0], None)
mesh.dmVars.addField(meshVar2.dm.getField(meshVar2.field_id)[0], None)
mesh.dmVars.addField(meshVar3.dm.getField(meshVar3.field_id)[0], None)

mesh.dm.addField(meshVar1.dm.getField(meshVar1.field_id)[0], None)
mesh.dm.addField(meshVar2.dm.getField(meshVar2.field_id)[0], None)
mesh.dm.addField(meshVar3.dm.getField(meshVar3.field_id)[0], None)

mesh.dm_h.addField(meshVar1.dm.getField(meshVar1.field_id)[0], None)
mesh.dm_h.addField(meshVar2.dm.getField(meshVar2.field_id)[0], None)
mesh.dm_h.addField(meshVar3.dm.getField(meshVar3.field_id)[0], None)



In [6]:
print(meshVar1._lvec)
print(meshVar2._lvec)

In [7]:
meshVar1._lvec.setArray(1)
meshVar2._lvec.setArray(2)

In [8]:
mesh.dmVars.createFieldDecomposition()

(['testVar1', 'testVar2', 'testVar3'],
  <petsc4py.PETSc.DMPlex at 0x10f0f19e0>])

In [9]:
mesh.dm_h.createDS()

mesh.dm_h.copyDS(mesh.dm_h.getCoarseDM())
cds = mesh.dm_h.getCoarseDM().getDS()


# mesh.dm_h.createFieldDecomposition()

TypeError: copyDisc() takes exactly 1 positional argument (0 given)

In [ ]:
dm0 = mesh.dm_h
for i in range(mesh.dm_h.getRefineLevel()):
        cdm = dm0.getCoarseDM()
        mesh.dm_h.copyFields(cdm)
        dm0 = cdm


In [ ]:
mesh.dm_h.createFieldDecomposition()

In [11]:
nvec = meshVar1._lvec.copy()

In [12]:
nvec = nvec.createNest((meshVar1._lvec, meshVar2._lvec), None, None)

In [13]:
nvec.getNestSubVecs()

[<petsc4py.PETSc.Vec at 0x11b847c90>, <petsc4py.PETSc.Vec at 0x11b847c40>]

In [14]:
nvec.array

array([1., 1., 1., ..., 2., 2., 2.])